<h3>Pre-Requisite</h3>

In [8]:
!pip install tabulate
!pip install pandas
!pip install Pyarrow

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)



[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<h3>Dataset - Black Friday Sales</h3>

In [3]:
!wget -q https://www.dropbox.com/s/8y6a1zloiscuo5d/black_friday_sales.zip
!tar -xf black_friday_sales.zip

In [5]:
import pandas as pd

dataFile = 'train.csv'
df = pd.read_csv(dataFile)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


<h3>CSV Agent</h3>
<p>
NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.
</p>

In [13]:
from langchain_experimental.agents import create_csv_agent
from langchain_openai import OpenAI

agent = create_csv_agent(OpenAI(temperature=0), 
                         dataFile, 
                         verbose=False)
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [14]:
agent.invoke({"input": "How many rows are there?"})

{'input': 'How many rows are there?',
 'output': 'There are 550068 rows in the dataframe.'}

In [15]:
agent.invoke({"input":"How many females are there?"})

{'input': 'How many females are there?',
 'output': 'There are 135809 females in the dataframe.'}

In [16]:
agent.invoke({"input":"how many people have stayed more than 3 years in the city?"})

{'input': 'how many people have stayed more than 3 years in the city?',
 'output': '84726'}